In [1]:
!mkdir -p ~/agave/funwave-tvd-jenkins-pipeline

%cd ~/agave/funwave-tvd-jenkins-pipeline

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

!auth-tokens-refresh

/home/jovyan/agave/funwave-tvd-jenkins-pipeline
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Token for sandbox:stevenrbrandt successfully refreshed and cached for 14400 seconds
4478f4983138e6bab1740f311c36e6f


# Everything as Code
* Every piece of config, infrastructure, and software should be stored in VCS
* Single source of truth
* Reproducibility, versioning, change management
* Avoid config drift
* Everything is managed with the same practice and rigor as application code

## Managing third-party code
* Industry uses mono repos, which works well. But what if you don't control 100% of your code?
* Open science is built on open-source software
  * This means a lot of third-party dependencies that you don't have direct control of.
  * You can't fork everything, as it becomes a maintenance hassle and makes it difficult to contribute back upstream.
* **Solution:** Submodules, explicit version requirements, meaningful tagging schemes, and bringing it all together in a single application repo.


## Commit Our App Configuration
* From here on out, all changes will be made directly in the Funwave repo.
* Put our build app in a place that will be accessible alongside the code
* Build process and build app can now be managed alongside code, and treated as an aspect of development.

In [2]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b dev"

Switched to a new branch 'dev'
M	src/Makefile


## Branching, Versioning, Tagging Strategies

### GitFlow
* [GitFlow](https://datasift.github.io/gitflow/IntroducingGitFlow.html), a successful git branching workflow for teams of any size.

![The GitFlow Strategy](https://datasift.github.io/gitflow/GitFlowHotfixBranch.png)

### Semantic Versioning
* Components of a semantic version
* Why use a semantic versioning scheme?
  * Sets expectations
  * Communicates changes
  * Meaningful version flexibility for consumer

In [7]:
writefile("version.txt","""3
2
0
dev""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F version.txt /home/jovyan/FUNWAVE-TVD/
!ssh sandbox "git config --global user.email sbrandt@cct.lsu.edu"
!ssh sandbox "git config --global user.name 'Steven R. Brandt'"
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add version.txt && git commit -m 'Adding version tracker.' && git checkout -b build-automation"

Writing file `version.txt'
Uploading version.txt...
######################################################################## 100.0%
+ cd /home/jovyan/FUNWAVE-TVD
+ git add version.txt
+ git commit -m 'Adding version tracker.'
[dev 619fb0c] Adding version tracker.
 1 file changed, 4 insertions(+)
 create mode 100644 version.txt
+ git checkout -b build-automation
M	src/Makefile
Switched to a new branch 'build-automation'


# Improving Our App for Automation
* Restructure Dockerfile
  * Explicit commit requirements for FunWave, so that a single version of our Agave app will only refer to a single version of the code.
  * Copy repo into your Dockerfile, don't clone. This ensures that the Dockerfile cannot drift from the code version it is intended to match, and the two can reflect one another in git history.

In [11]:
writefile("Dockerfile","""
FROM stevenrbrandt/science-base
MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>

ARG BUILD_DATE
ARG VERSION

LABEL org.agaveplatform.ax.architecture="x86_64"                                \
      org.agaveplatform.ax.build-date="\$BUILD_DATE"                             \
      org.agaveplatform.ax.version="\$VERSION"                             \
      org.agaveplatform.ax.name="${AGAVE_USERNAME}/funwave-tvd"    \
      org.agaveplatform.ax.summary="Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi." \
      org.agaveplatform.ax.vcs-type="git"                                       \
      org.agaveplatform.ax.vcs-url="https://github.com/fengyanshi/FUNWAVE-TVD" \
      org.agaveplatform.ax.license="BSD 3-clause"
      
USER root
RUN mkdir -p /home/install/FUNWAVE-TVD
RUN chown jovyan /home/install/FUNWAVE-TVD
COPY --chown jovyan ./* /home/install/FUNWAVE-TVD/
USER jovyan

WORKDIR /home/install/FUNWAVE-TVD/src
RUN perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

RUN mkdir -p /home/jovyan/rundir
WORKDIR /home/jovyan/rundir
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Dockerfile /home/jovyan/FUNWAVE-TVD/
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add Dockerfile"

Writing file `Dockerfile'
Uploading Dockerfile...
######################################################################## 100.0%
+ cd /home/jovyan/FUNWAVE-TVD
+ git add Dockerfile


In [19]:
writefile("funwave-build-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

sudo docker build \
    --build-arg "BUILD_DATE=\${AGAVE_JOB_SUBMIT_TIME}" \
    --build-arg "VERSION=\${VERSION}" \
    --rm -t funwave-tvd:\${VERSION} .

docker inspect funwave-tvd:\${VERSION}

""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/FUNWAVE-TVD/
!ssh sandbox "cd ~/FUNWAVE-TVD && git add funwave-build-wrapper.txt"

Writing file `funwave-build-wrapper.txt'
Uploading funwave-build-wrapper.txt...
######################################################################## 100.0%


In [16]:
writefile("funwave-build-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Builds the funwave docker image",
   "shortDescription":"Funwave docker build",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"automation/funwave-tvd-docker-automation",
   "templatePath":"funwave-build-wrapper.txt",
   "testPath":"test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/FUNWAVE-TVD/
!ssh sandbox "set  -x && cd ~/FUNWAVE-TVD && git add funwave-build-wrapper.txt"

Writing file `funwave-build-app.txt'
Uploading funwave-build-wrapper.txt...
######################################################################## 100.0%
+ cd /home/jovyan/FUNWAVE-TVD
+ git add funwave-build-wrapper.txt


# Build Automatically On Each Commit
* Intro to Jenkins
  * What is it?
  * What is its primary use case?
  * Deciding if it the right solution for you. What are alternatives?

## Components of a Build Pipeline
* Build environment (build app you just made)
* Jenkinsfile, automating your build
* Triggers, starting your build
* Tests, validating your build
* Secrets, and site-specific configuration

## Modularize Our Config
* Right now the build app is set up to configure from environment variables not accessible to the Jenkins host
* We need to make this environment accessible to the build pipline, while also making it easy to understand and maintain

### Strategies for Maintaining Credentials and Config
* Travis CI encrypted values
* Jenkins Freestyle job, injecting config as environment variables known only to build host
* Consider your audience. How is your code meant to be used?
  * Is it OSS?
  * Do you want to encourage reuse?
  * Will it be stored publicly or privately?
  * Where will it be built?
* **Admin Consideration:** Vault, secrets stores
* For simplicity, we will simply include config in the Groovy script.

## Create Our Jenkins Pipeline
* What is a Jenkins pipeline?
* Purpose of a Jenkinsfile
* How is it read by Jenkins?
* Freestyle, Pipeline, Blue Ocean
* Navigate to https://<username\>.sc18.training.agaveplatform.org/jenkins/job to view the associated pipeline
* What are the different components of this job?
  * SCM
  * Jenkinsfile location
  * Triggers
  * Branches, touch on concurrent builds
  * Notification

### Creating A Jenkins Pipeline


In [18]:
writefile(".jenkinsfile","""
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F .jenkinsfile /home/jovyan/FUNWAVE-TVD/
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add .jenkinsfile"

Writing file `.jenkinsfile'
Uploading .jenkinsfile...
######################################################################## 100.0%
+ cd /home/jovyan/FUNWAVE-TVD
+ git add .jenkinsfile


### Triggering our Jenkins Pipeline From a Commit
* Polling, webhooks, git hooks
* We'll use a post-commit git hook for simplicity.
* What are git hooks?

In [21]:
!env

MINICONDA_VERSION=4.3.21
MAIL=/var/mail/jovyan
USER=jovyan
LANGUAGE=en_US.UTF-8
AGAVE_TENANTS_API_BASEURL=https://sandbox.agaveplatform.org/tenants
MPLBACKEND=module://ipykernel.pylab.backend_inline
AGAVE_CACHE_DIR=/home/jovyan/work/.$AGAVE_TENANT
HOSTNAME=6e57ea83e04c
VM_IPADDRESS=52.15.183.149
AGAVE_STORAGE_HOME_DIR=/home/jovyan
SHLVL=1
XDG_CACHE_HOME=/home/jovyan/.cache/
HOME=/home/jovyan
NGROK_TOKEN=nkupApaEBZofn6LXd82_3XFuXeDBEJwvQJrh89JbB
ENVIRONMENT=training
VM_PORT=10022
PAGER=cat
AGAVE_SYSTEM_HOST=0.tcp.ngrok.io
MACHINE_NAME=sandbox
NB_USER=jovyan
AGAVE_PASSWORD=GlaringChorus059
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
AGAVE_CLI_COMPLETION_SHOW_FILES=no
GRANT_SUDO=yes
LOGNAME=jovyan
_=/usr/bin/env
USE_TUNNEL=True
AGAVE_SYSTEM_PORT=19781
TERM=xterm-color
MACHINE_USERNAME=jovyan
AGAVE_CLI_COMPLETION_CACHE_LIFETIME=0
ROTATE=yes
AGAVE_CLI_COMPLETION_SHOW_FILE_PATHS=no
PATH=/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/jq/bin:/h

In [27]:
# Post-commit hook to trigger Jenkins job via curl
setvar("""
JENKINS_JOB_URL="https://${AGAVE_USERNAME}.sc18.training.agaveplatform.org/jenkins/job"
JENKINS_JOB_NAME="funwave-build-pipeline"
BUILD_TOKEN="sc18-training-job"
""")
writefile("post-commit","""#!/bin/bash
JENKINS_JOB_URL="${JENKINS_JOB_URL}"
JEKNINS_JOB_NAME="${JENKINS_JOB_NAME}"
BUILD_TOKEN="${BUILD_TOKEN}"
curl -sk "$JENKINS_JOB_URL/$JEKNINS_JOB_NAME/build?token=$BUILD_TOKEN"
""")
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/.git/hooks
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F post-commit /home/jovyan/FUNWAVE-TVD/.git/hooks/
# Set the hook to executable so that it won't be ignored.
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD/ && git add .git/hooks/post-commit && git update-index --chmod=+x post-commit"

JENKINS_JOB_URL="https://stevenrbrandt.sc18.training.agaveplatform.org/jenkins/job"
JENKINS_JOB_NAME="funwave-build-pipeline"
BUILD_TOKEN="sc18-training-job"
Writing file `post-commit'
Successfully created folder /home/jovyan/FUNWAVE-TVD/.git/hooks
Uploading post-commit...
######################################################################## 100.0%
+ cd /home/jovyan/FUNWAVE-TVD/
+ git add .git/hooks/post-commit
error: Invalid path '.git/hooks/post-commit'
error: unable to add .git/hooks/post-commit to index
fatal: adding files failed


# Commit Your Code, Start Your Pipeline
* We'll commit our changes and merge our feature branch back into `dev`
* This will prompt the pipeline to start
* Watch the pipeline run: https://<username\>.sc18.training.agaveplatform.org/jenkins/job
* In practice, merge requests should prompt two builds:
  * One to validate the feature branch itself
  * Another to validate that the build still passes after the merge
* In the interest of time, we will only validate individual commits to the `dev` branch.

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added jenkinsfile and post-commit hook.'"
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout dev && git merge --squash build-automation"

# Validating Your Build
* Automation is great, but things can quickly go wrong.
* This is why CI/CD strongly emphasizes good testing practices.
* Testing is central to CI/CD
  * It allows you assess viability of each commit
  * It is the determination of whether or not code can be successfully integrated
  * It allows for code to be automatically deployed to prod, without the direct oversight of a tightly controlled group of developers.
* Production pipelines have multiple testing guards
  * Types of tests: Unit, Functional, Acceptance, Benchmarks, ...

## Adding a Benchmark
* Let's add a simple benchmark to validate performance after each build.
* We'll make a new feature branch for this benchmark

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b benchmark"

### Update the Jenkinsfile
* We need to add a new stage to the Jenkins pipeline to run the benchmark if the preceeding stages of the build executed successfully.

In [ ]:
writefile(".jenkinsfile","""
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F .jenkinsfile /home/jovyan/FUNWAVE-TVD/.jenkinsfile

### Merge Your Benchmark
* Let's merge our benchmark back into the `dev` branch and watch it run!

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added benchmark to pipeline.'"
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout dev && git merge --squash benchmark"

# Continuous Delivery and Deployment
* Once your build pipeline has run, and your changeset has passed all testing guards, what do you do?
* Continuous Delivery:
  * Package and publish your code
  * Singularity Registry, DockerHub, etc..
* Continuous Deployment:
  * Package your code and deploy it to pre-prod or prod servers.